<a href="https://colab.research.google.com/github/nmonson1/mnist_exploration/blob/main/Copy_of_Hessian_of_MNIST_Classifier_with_PyTorch_SERI_MATS_Winter_Cohort_2022_John.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Classifier with PyTorch - Finding the Hessian

## Importing dependencies

In [ ]:
import time
import math

import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from PIL import Image

In [ ]:
x=torch.tensor([2,3,4])
y=torch.tensor([1,2,3])
z=x+y
z=2*z
print(z.grad_fn)

None


## Neural Network

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 1, 3, 1)
    self.fc2 = nn.Linear(169, 10)
    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
      module.weight.data.normal_(mean=0.0, std=0.5)
      if module.bias is not None:
        module.bias.data.zero_()

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = torch.flatten(x, 1)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=1)
    return output

## Hyperparameters

In [ ]:
BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000
EPOCHS = 10
LR = 1.0
GAMMA = 0.7
num_params = 1710

In [ ]:
use_cuda = torch.cuda.is_available()

if use_cuda:
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

## Creating model, loss function, and optimizer

In [ ]:
model = Net().to(DEVICE)
optimizer = optim.Adadelta(model.parameters(), lr=LR)

loss_fn = nn.CrossEntropyLoss()

## Loading data

In [ ]:
train_kwargs = {'batch_size': BATCH_SIZE}
test_kwargs = {'batch_size': TEST_BATCH_SIZE}
if use_cuda:
  cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
  train_kwargs.update(cuda_kwargs)
  test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(Subset(dataset1, np.random.randint(len(dataset1), size=1500)), **train_kwargs)  # subset of datasets to keep network overparameterized
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

## Train and test functions

In [ ]:
def train(dry_run: bool, model, device, train_loader, optimizer, epoch):
  model.train()

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)

    loss.backward()
    optimizer.step()

    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
      if dry_run:
        break


def test(model, device, test_loader):
  model.eval()

  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
      pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

## Training the model

In [ ]:
scheduler = StepLR(optimizer, step_size=1, gamma=GAMMA)
for epoch in range(1, EPOCHS + 1):
  train(False, model, DEVICE, train_loader, optimizer, epoch)
  test(model, DEVICE, test_loader)
  scheduler.step()

torch.save(model.state_dict(), "mnist_cnn_small.pt")

Train Epoch: 1 [0/1500 (0%)]	Loss: 14.053327
Train Epoch: 1 [640/1500 (42%)]	Loss: 7.843325
Train Epoch: 1 [1280/1500 (83%)]	Loss: 6.455245

Test set: Average loss: 5.0164, Accuracy: 1221/10000 (12%)

Train Epoch: 2 [0/1500 (0%)]	Loss: 5.542170
Train Epoch: 2 [640/1500 (42%)]	Loss: 5.029972
Train Epoch: 2 [1280/1500 (83%)]	Loss: 3.821646

Test set: Average loss: 3.9806, Accuracy: 1480/10000 (15%)

Train Epoch: 3 [0/1500 (0%)]	Loss: 4.083921
Train Epoch: 3 [640/1500 (42%)]	Loss: 3.941819
Train Epoch: 3 [1280/1500 (83%)]	Loss: 3.859973

Test set: Average loss: 3.6049, Accuracy: 1703/10000 (17%)

Train Epoch: 4 [0/1500 (0%)]	Loss: 3.851332
Train Epoch: 4 [640/1500 (42%)]	Loss: 3.697637
Train Epoch: 4 [1280/1500 (83%)]	Loss: 3.568192

Test set: Average loss: 3.4062, Accuracy: 1856/10000 (19%)

Train Epoch: 5 [0/1500 (0%)]	Loss: 3.473230
Train Epoch: 5 [640/1500 (42%)]	Loss: 3.070641
Train Epoch: 5 [1280/1500 (83%)]	Loss: 3.483856

Test set: Average loss: 3.2768, Accuracy: 1952/10000 (20%)


## Hessian function

In [ ]:
def get_hessian(model: nn.Module, loss: torch.Tensor):
  # Hessian of the loss w.r.t. the parameters

  optimizer = optim.Adadelta(model.parameters(), lr=1)
  model = model.to(DEVICE)
  model.eval()
  optimizer.zero_grad()

  xs = optimizer.param_groups[0]['params']
  ys = loss

  grads = torch.autograd.grad(ys.to(DEVICE), xs, create_graph=True)

  grads2 = []
  for j, grad in enumerate(grads):
    start = time.time()
    print(f"Calculating pairwise (every param, every layer) second-order derivatives for params of layer {j}...")

    grad = torch.reshape(grad, [-1])

    grads2_tmp = []
    for count, g in enumerate(grad):
      percent_done = count / (len(grad))
      if (count % 100 == 0):
        print(f"{percent_done:.2%} done...")

      grads2_tmp_tmp = []  # creating variable naming, innit?
      for x in xs:
        g2 = torch.autograd.grad(g, x, retain_graph=True)[0]
        g2 = torch.reshape(g2, [-1])
        grads2_tmp_tmp.append(g2.data.cpu())

      flat = None
      for second_orders in grads2_tmp_tmp:
        if(flat is None):
          flat = second_orders.flatten()
        else:
          flat = torch.concat((flat, second_orders.flatten()))

      grads2_tmp.append(flat.numpy())
    grads2 += grads2_tmp

    print('Time used is ', time.time() - start)
  return np.array(grads2)

## Hessian over the training dataset

In [ ]:
losses = []
for i, (data, targets) in enumerate(train_loader):
  print(f"Computing average loss on train set {(i+1)/len(train_loader):.2%}   ", end='\r')
  data = data.to(DEVICE)
  targets = targets.to(DEVICE)

  losses.append(loss_fn(model(data), targets))

avg_loss = torch.mean(torch.stack(losses))
print(f'Average Train Loss: {avg_loss.item()} \n')

hessian = get_hessian(model, avg_loss)

Average Train Loss: 2.9914748668670654 

Calculating pairwise (every param, every layer) second-order derivatives for params of layer 0...
0.00% done...
Time used is  0.5287890434265137
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 1...
0.00% done...
Time used is  0.045325517654418945
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 2...
0.00% done...
5.92% done...
11.83% done...
17.75% done...
23.67% done...
29.59% done...
35.50% done...
41.42% done...
47.34% done...
53.25% done...
59.17% done...
65.09% done...
71.01% done...
76.92% done...
82.84% done...
88.76% done...
94.67% done...
Time used is  152.2990574836731
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 3...
0.00% done...
Time used is  0.3349490165710449


In [ ]:
hessian[:4, :4]  # showing parts of the Hessian

array([[14.285873 ,  9.79766  ,  6.301756 ,  8.839961 ],
       [ 9.79766  ,  9.980309 ,  7.5023203,  5.972213 ],
       [ 6.301756 ,  7.5023203,  9.037817 ,  3.3699284],
       [ 8.83996  ,  5.972213 ,  3.3699286,  9.470606 ]], dtype=float32)

## Determinant of Hessian

In [ ]:
torch.det(torch.Tensor(hessian))

tensor(-0.)

# Now with (L2) regularization!



In [ ]:
model_l2 = Net().to(DEVICE)
optimizer_l2 = optim.Adadelta(model_l2.parameters(), lr=LR, weight_decay=1e-5)

scheduler = StepLR(optimizer, step_size=1, gamma=GAMMA)
for epoch in range(1, EPOCHS + 1):
  train(False, model_l2, DEVICE, train_loader, optimizer_l2, epoch)
  test(model_l2, DEVICE, test_loader)
  scheduler.step()

torch.save(model_l2.state_dict(), "mnist_cnn_small_l2.pt")

Train Epoch: 1 [0/1500 (0%)]	Loss: 7.257487
Train Epoch: 1 [640/1500 (42%)]	Loss: 4.341743
Train Epoch: 1 [1280/1500 (83%)]	Loss: 3.155448

Test set: Average loss: 3.0490, Accuracy: 1153/10000 (12%)

Train Epoch: 2 [0/1500 (0%)]	Loss: 3.053784
Train Epoch: 2 [640/1500 (42%)]	Loss: 3.006741
Train Epoch: 2 [1280/1500 (83%)]	Loss: 2.776328

Test set: Average loss: 2.5493, Accuracy: 1883/10000 (19%)

Train Epoch: 3 [0/1500 (0%)]	Loss: 2.250318
Train Epoch: 3 [640/1500 (42%)]	Loss: 2.344134
Train Epoch: 3 [1280/1500 (83%)]	Loss: 2.425436

Test set: Average loss: 2.2084, Accuracy: 2586/10000 (26%)

Train Epoch: 4 [0/1500 (0%)]	Loss: 2.450996
Train Epoch: 4 [640/1500 (42%)]	Loss: 2.018063
Train Epoch: 4 [1280/1500 (83%)]	Loss: 2.030682

Test set: Average loss: 1.9338, Accuracy: 3345/10000 (33%)

Train Epoch: 5 [0/1500 (0%)]	Loss: 2.039955
Train Epoch: 5 [640/1500 (42%)]	Loss: 1.861044
Train Epoch: 5 [1280/1500 (83%)]	Loss: 1.688241

Test set: Average loss: 1.6923, Accuracy: 4324/10000 (43%)



In [ ]:
losses_l2 = []
for i, (data, targets) in enumerate(train_loader):
  print(f"Computing average loss on train set {(i+1)/len(train_loader):.2%}   ", end='\r')
  data = data.to(DEVICE)
  targets = targets.to(DEVICE)

  losses_l2.append(loss_fn(model_l2(data), targets))

avg_loss_l2 = torch.mean(torch.stack(losses_l2))
print(f'Average Train Loss (L2 Regularized): {avg_loss_l2.item()} \n')

hessian_l2 = get_hessian(model_l2, avg_loss_l2)

Average Train Loss (L2 Regularized): 0.9927076101303101 

Calculating pairwise (every param, every layer) second-order derivatives for params of layer 0...
0.00% done...
Time used is  0.5337684154510498
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 1...
0.00% done...
Time used is  0.04964184761047363
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 2...
0.00% done...
5.92% done...
11.83% done...
17.75% done...
23.67% done...
29.59% done...
35.50% done...
41.42% done...
47.34% done...
53.25% done...
59.17% done...
65.09% done...
71.01% done...
76.92% done...
82.84% done...
88.76% done...
94.67% done...
Time used is  152.73879718780518
Calculating pairwise (every param, every layer) second-order derivatives for params of layer 3...
0.00% done...
Time used is  0.33493995666503906


In [ ]:
hessian_l2[:4, :4]

array([[7.906001 , 6.4523478, 4.5737157, 2.36314  ],
       [6.4523478, 7.199847 , 5.814742 , 2.117704 ],
       [4.5737157, 5.8147426, 6.8861847, 1.4781702],
       [2.3631403, 2.117704 , 1.4781703, 1.3852652]], dtype=float32)

In [ ]:
torch.det(torch.Tensor(hessian_l2))

tensor(-0.)

## Calculating eigenvalues of the L2 Hessian

Some ask why.  We ask why not?

In [ ]:
u, s, vh = np.linalg.svd(hessian_l2, full_matrices=False)

In [ ]:
(s < 0).any()

False

# Basin Volume

Refer [this post](https://www.lesswrong.com/posts/QPqztHpToij2nx7ET/hessian-and-basin-volume) for context.

## Log of volume of unit n-ball

We're using [Stirling's formula for approximating the gamma function](https://en.wikipedia.org/wiki/Volume_of_an_n-ball#Approximation_for_high_dimensions) to not have to deal with very large factorials here from the large number of dimensions.

In [ ]:
log_v = math.log(1 / math.sqrt(num_params * np.pi)) + num_params / 2 * math.log(2 * np.pi * np.e / num_params)  # calculating volume directly gives us 0

## Log of threshold term

In [ ]:
threshold = 2

In [ ]:
log_threshold_term = num_params / 2 * math.log(2 * threshold)

## L2 Regularization term

In [ ]:
wd = 1e-5  # weight decay
k = 1
init_std = 0.5
c = k / (init_std ** 2)

reg_term = torch.mul(wd + c, torch.eye(num_params))

## Log of denominator

In [ ]:
hessian_tmp = np.add(hessian_l2, reg_term)
log_determinant = torch.logdet(hessian_tmp)  # calculating determinant directly give us infinity

In [ ]:
log_denom = log_determinant * 1/2  # 1/2 * log(det) = log(det^1/2)

## Calculating (log of) basin volume

In [ ]:
print("Log volume of unit ball: {}".format(log_v))
print("Log of threshold term: {}".format(log_threshold_term))
print("Log of denominator: {}".format(log_denom))

Log volume of unit ball: -3942.742192807366
Log of threshold term: 1185.2816787575064
Log of denominator: 1190.5987548828125


In [ ]:
log_v_basin = log_v + log_threshold_term - log_denom

In [ ]:
log_v_basin

tensor(-3948.0591)